<a href="https://colab.research.google.com/github/marek-bardonski/airev-advanced-workshops/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# AI REV LLC - Copyrights 2020
#Change runtime mode to GPU

!nvcc --version
!pip3 install wget

# https://github.com/rapidsai/cudf/issues/3390
!pip3 install pyarrow==0.15.0 ## Workaround to cover up for Google Colab bug

#!pip3 install cudf-cuda100
#!pip3 install nvstrings-cuda100
#!pip3 install cuml-cuda100
#!pip3 install nvvm-cuda100

# intall miniconda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

# install RAPIDS packages
!conda install -q -y --prefix /usr/local -c conda-forge \
  -c rapidsai-nightly/label/cuda10.0 -c nvidia/label/cuda10.0 \
  cudf cuml

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
--2019-12-26 15:57:32--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745

Remember to restart the runtime at this moment.

In [15]:
!ls /env/python/

ls: cannot access '/env/python/': No such file or directory


In [0]:
# set environment vars
import sys, os, shutil
sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

# copy .so files to current working dir
for fn in ['libcudf.so', 'librmm.so']:
  shutil.copy('/usr/local/lib/'+fn, os.getcwd())

In [0]:
import wget
from zipfile import ZipFile
import os
import cudf
import sys, os
import nvcategory
import os
import numpy as np
#import cuml (bug https://gitmemory.com/issue/rapidsai/cuml/404/477551898)
import nvstrings
import nltk
from numba import cuda
import json

     |████████████████████████████████| 60.5MB 51kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [5]:
# Raw datasets are samples from WebHose.io 
print('Beginning dataset download with wget module')

url = 'https://bardonski.pl/chineese.zip'
wget.download(url)
url = 'https://bardonski.pl/arabic.zip'
wget.download(url)

Beginning dataset download with wget module


'arabic.zip'

In [0]:
zip = ZipFile('chineese.zip')
zip.extractall()

!mkdir chineese

zip = ZipFile('630_webhose-2016-10_20170904084325.zip')
zip.extractall('chineese')

In [0]:
zip = ZipFile('arabic.zip')
zip.extractall()

!mkdir arabic

zip = ZipFile('627_webhose-2016-10_20170904083346.zip')
zip.extractall('arabic')

In [8]:
#Expected 236384 arabic articles
#Expected 316004 chineese articles
!ls arabic -l | wc -l
!ls chineese -l | wc -l


236384
316004


In [9]:
def get_text(lines):
    """
        returns non empty lines from a list of lines
    """
    decoded = json.loads(lines[0])
    clean_lines = decoded['text']
    return [clean_lines]

def get_txt_lines(data_dir):
    """
        Read text lines from gutenberg tests
        returns (text_ls,fname_ls) where 
        text_ls = input_text_lines and fname_ls = list of file names
    """
    text_ls = []
    fname_ls = []
    for fn in os.listdir(data_dir):
        full_fn = os.path.join(data_dir,fn)
        with open(full_fn,encoding="utf-8",errors="ignore") as f:
            content = f.readlines()
            content = get_text(content)
            if content is not None:
                text_ls += content
                ### dont add .txt to the file
                fname_ls += [fn[:-4]]*len(content)
        #return text_ls, fname_ls    
    
    return text_ls, fname_ls    
    
print("File Read Time:")
%time txt_ls,fname_ls = get_txt_lines('arabic')
df = cudf.DataFrame()

print("\nCUDF  Creation Time:")
%time df['text'] = nvstrings.to_device(txt_ls)

File Read Time:
CPU times: user 12.6 s, sys: 2.95 s, total: 15.6 s
Wall time: 15.7 s

CUDF  Creation Time:
CPU times: user 2.45 s, sys: 932 ms, total: 3.38 s
Wall time: 3.46 s


In [10]:
print("Number of lines in the DF = {:,}".format(len(df)))

Number of lines in the DF = 236,383


In [11]:
df.head(10).to_pandas()

,text
0,مارون: مهند عندو عقدي ضاربتو شمس هو وصغير ملفا...
1,لهذا السبب.. خالد الصاوي يوجه رسالة شكر لجمهور...
2,انتونيو غوتيريس يصبح رسميًا الأمين العام الجدي...
3,رام الله- شاشة نيوز- تتقدم اسرة شاشة نيوز باحر...
4,الليثي ضحايا الدجل والشعوذة يتعرضن للإعتداء ال...
5,أعلن مسئولون باكستانيون ارتفاع حصيلة ضحايا اله...
6,قرعة سهلة لأندية القمة في قرعة دور الـ32 بكأس ...
7,صورة «سيلفي» كادت تدخل نجم البوب الأميركي جا...
8,بدد باحثون من جامعة أوكسفورد البريطانية أسطورة...
9,قفزت أسعار النفط ثلاثة بالمائة يوم أمس الاثنين...


In [12]:
# remove the following punctuations/characters from cudf
filters = [ '!', '"', '#', '$', '%', '&', '(', ')', '*', '+', '-', '.', '/',  '\\', ':', ';', '<', '=', '>',
           '?', '@', '[', ']', '^', '_', '`', '{', '|', '}', '\~', '\t','\\n',"'",",",'~' , '—']


text_col_sample = df.head(5)
text_col_sample['text'].to_pandas()

0    مارون: مهند عندو عقدي ضاربتو شمس هو وصغير ملفا...
1    لهذا السبب.. خالد الصاوي يوجه رسالة شكر لجمهور...
2    انتونيو غوتيريس يصبح رسميًا الأمين العام الجدي...
3    رام الله- شاشة نيوز- تتقدم اسرة شاشة نيوز باحر...
4    الليثي ضحايا الدجل والشعوذة يتعرضن للإعتداء ال...
Name: text, dtype: object

In [14]:
text_col_sample['text_clean'] = text_col_sample['text'].str.replace_multi(filters, ' ', regex=False)
text_col_sample['text_clean'].to_pandas()

0    مارون  مهند عندو عقدي ضاربتو شمس هو وصغير ملفا...
1    لهذا السبب   خالد الصاوي يوجه رسالة شكر لجمهور...
2    انتونيو غوتيريس يصبح رسميًا الأمين العام الجدي...
3    رام الله  شاشة نيوز  تتقدم اسرة شاشة نيوز باحر...
4    الليثي ضحايا الدجل والشعوذة يتعرضن للإعتداء ال...
Name: text_clean, dtype: object

In [0]:
text_col_sample['text_clean'] = text_col_sample['text_clean'].str.lower()

In [0]:
nltk.download('stopwords')
STOPWORDS = nltk.corpus.stopwords.words('arabic')
STOPWORDS = nvstrings.to_device(STOPWORDS)

In [0]:
text_col_sample['text_clean'] = nvtext.replace_tokens(text_col_sample['text_clean'].data, STOPWORDS, ' ')
text_col_sample['text_clean'].to_pandas()

In [0]:
import nvtext

In [0]:
!ls /usr/local/cuda-10.1/